In [1]:
from agent import Agent
from referee.game import PlayerColor
agent = Agent(PlayerColor.RED)

agent

Testing: I am playing as red


In [2]:
action = agent.action()
action

SpawnAction(cell=HexPos(r=3, q=3))

In [8]:
from agent.board import MatrixBoard
import numpy as np

matrix = np.zeros([2,7,7], dtype=int)
board = MatrixBoard(matrix)

In [9]:
from referee.game import \
    PlayerColor, Action, SpawnAction, SpreadAction, HexPos, HexDir

board.apply_action(SpawnAction(HexPos(1,1)), PlayerColor.RED)

In [10]:
print(board.render(use_color=True, use_unicode=True))

                         ..     
                     ..      ..     
                 ..      ..      ..     
             ..      ..      ..      ..     
         ..      ..      ..      ..      ..     
     ..      ..      ..      ..      ..      ..     
 ..      r1      ..      ..      ..      ..      ..     
     ..      ..      ..      ..      ..      ..     
         ..      ..      ..      ..      ..     
             ..      ..      ..      ..     
                 ..      ..      ..     
                     ..      ..     
                         ..     

